YAHOO!ファイナンスのサイトから株価のリアルタイム情報をスクレイピングする

取りたい銘柄のコードを「getdata.csv」で用意してファイルに置く（毎回消える）

In [ ]:
#csvの読み込み処理
import csv

#配列宣言
getdata_list = []

#csvファイルを指定
filename = 'getdata.csv'

#csvファイルを読み込み
with open(filename) as f:
  reader = csv.reader(f)
  #csvファイルのデータをループ
  for row in reader:
    #A列を配列へ格納
    getdata_list.append(str(row[0]).replace("\ufeff",""))
print(getdata_list)

['7203.T', '7267.T', '7201.T', '7261.T']


In [ ]:
#配列を1個ずつ繰り返す
#各種宣言
import urllib.parse
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#データ格納用の配列を用意
company_name_list = []
kabuka_list = []
kabuka_zenzitsu_list = []
kabuka_zo_list = []
takane_list = []
yasune_list = []
dekidaka_list = []

In [ ]:
for g_list in getdata_list:
  #URLの結合を行う（各処理に対して、ページを行き来きする必要があるため）
  load_url = urllib.parse.urljoin("https://finance.yahoo.co.jp/quote/",g_list)
  print(load_url)

  #下記よりスクレイピングを行う
  html = requests.get(load_url)
  soup = BeautifulSoup(html.content, "html.parser")
  #確認のための表示
  #print(soup)

  #初期化
  ck_page = ''
  #会社名の取得
  for element in soup.find_all(class_="_6uDhA-ZV"):
    ck_page = element.text
    if ck_page != '':
      company_name_list.append(ck_page)
      print(company_name_list)

  #カウント用の変数を用意
  count = 0
  #株価関連データの取得
  for element in soup.find_all(class_="_3rXWJKZF"):
    #　　関係のあるデータだけを絞るためカウント用の変数を作成し、使用する
    if count == 0:
      kakaku = element.text
      if kakaku != '':
        kabuka_list.append(kakaku)
      print(kabuka_list)
    #前日比のデータは２番目
    elif count == 1:
      kabuka_zenzitsu_list.append(element.text)
      print(kabuka_zenzitsu_list)
    #前日終値のデータは4番目
    elif count == 3:
      kabuka_zo_list.append(element.text)
      print(kabuka_zo_list)
    #高値のデータは6番目
    elif count == 5:
      takane_list.append(element.text)
      print(takane_list)
    #安値のデータは7番目
    elif count == 6:
      yasune_list.append(element.text)
      print(yasune_list)
    #出来高のデータは８番目
    elif count == 7:
      dekidaka_list.append(element.text)
      print(dekidaka_list)
    count += 1
  #何もない場合はから文字列
  if ck_page == '':
    #会社名のみ該当列があるか否かを表記
    company_name_list.append('該当コードなし')
    kabuka_list.append('')
    kabuka_zenzitsu_list.append('')
    kabuka_zo_list.append('')
    takane_list.append('')
    yasune_list.append('')
    dekidaka_list.append('')

#データフレームに格納
col_main = ["東証４桁コード","会社名","株価","前日比","前日終値","高値","安値","出来高"]
df_list = [[getdata_list[i],company_name_list[i],kabuka_list[i],kabuka_zenzitsu_list[i],kabuka_zo_list[i],takane_list[i],
            yasune_list[i], dekidaka_list[i],]
            for i in range(len(getdata_list))]
df = pd.DataFrame(data=df_list,columns=col_main)

https://finance.yahoo.co.jp/quote/7203.T
['トヨタ自動車(株)']
['1,796']
['-3']
['1,799']
['1,819.5']
['1,793.5']
['11,345,600']
https://finance.yahoo.co.jp/quote/7267.T
['トヨタ自動車(株)', 'ホンダ']
['1,796', '3,055']
['-3', '+13']
['1,799', '3,042']
['1,819.5', '3,103']
['1,793.5', '3,046']
['11,345,600', '2,217,400']
https://finance.yahoo.co.jp/quote/7201.T
['トヨタ自動車(株)', 'ホンダ', '日産自動車(株)']
['1,796', '3,055', '414.6']
['-3', '+13', '+2.2']
['1,799', '3,042', '412.4']
['1,819.5', '3,103', '418.3']
['1,793.5', '3,046', '413.3']
['11,345,600', '2,217,400', '6,423,300']
https://finance.yahoo.co.jp/quote/7261.T
['トヨタ自動車(株)', 'ホンダ', '日産自動車(株)', 'マツダ(株)']
['1,796', '3,055', '414.6', '970']
['-3', '+13', '+2.2', '-5']
['1,799', '3,042', '412.4', '975']
['1,819.5', '3,103', '418.3', '986']
['1,793.5', '3,046', '413.3', '968']
['11,345,600', '2,217,400', '6,423,300', '1,769,000']


In [ ]:
#EXCELとしてアウトプット
df.to_excel("result.xlsx")